# データセットの準備

In [1]:
import pandas as pd
import numpy as np

## Japanese Realistic Textual Entailment Corpus

データセットをダウンロードします。
データセットのラインセスについては
[データセット情報](dataset_info.md)
を確認してください。

データセットのリポジトリをclone をします。

```sh
$ git clone https://github.com/megagonlabs/jrte-corpus
```

## データのロード

In [2]:
pn_df = pd.read_table(
    "jrte-corpus/data/pn.tsv",
    header=None,
    names=["id", "label", "text", "judges", "usage"]
)

In [3]:
pn_df.head()

,id,label,text,judges,usage
0,pn17q00001,0,出張でお世話になりました。,"{""0"": 3}",test
1,pn17q00002,0,朝食は普通でした。,"{""0"": 3}",test
2,pn17q00003,1,また是非行きたいです。,"{""1"": 3}",test
3,pn17q00004,1,また利用したいと思えるホテルでした。,"{""1"": 3}",test
4,pn17q00005,1,駅から近くて便利でした。,"{""0"": 1, ""1"": 2}",test


## ラベルの変換

In [4]:
pn_df["label"] = pn_df["label"].map({1: "positive", 0: "neutral", -1: "negative"})

In [5]:
pn_df.head()

,id,label,text,judges,usage
0,pn17q00001,neutral,出張でお世話になりました。,"{""0"": 3}",test
1,pn17q00002,neutral,朝食は普通でした。,"{""0"": 3}",test
2,pn17q00003,positive,また是非行きたいです。,"{""1"": 3}",test
3,pn17q00004,positive,また利用したいと思えるホテルでした。,"{""1"": 3}",test
4,pn17q00005,positive,駅から近くて便利でした。,"{""0"": 1, ""1"": 2}",test


## データの保存

In [6]:
# 保存用のディレクトリを作成します
import pathlib

pathlib.Path("input").mkdir(exist_ok=True)

In [7]:
save_columns = ["label", "text", "judges"]
pn_df[save_columns].to_csv("input/pn.csv", index=None)

後のモデル学習の際に利用するために、ジャッジが一致しているサンプルのみを取り出した結果も保存しておきます。

In [8]:
import json

judges_all_same = pn_df["judges"] \
    .apply(lambda x: len(json.loads(x).keys()) == 1)

pn_df[judges_all_same] \
    .reset_index(drop=True)[save_columns] \
    .to_csv("input/pn_same_judge.csv", index=None)

## 二値分類モデルの学習データの作成

二値分類モデルの学習データの例としてすぐに使えるように、テキストに前処理を行った結果も保存しておきます。

In [9]:
# positive, neutral, negative の中から negative なレビューを当てるタスクとして、
# negativeを1に、それ以外のpositive, neutralを0に設定します。
pn_df["label_num"] = pn_df["label"].map({"positive": 0, "neutral": 0, "negative": 1})

In [10]:
import spacy

nlp = spacy.load("ja_core_news_md")

def tokenize(text):
   return [token.lemma_ for token in nlp(text)]

2022-05-27 01:08:58.535669: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-27 01:08:58.535760: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
pn_df["tokens"] = pn_df["text"].apply(lambda x: " ".join(tokenize(x)))

In [12]:
pn_df.head()

,id,label,text,judges,usage,label_num,tokens
0,pn17q00001,neutral,出張でお世話になりました。,"{""0"": 3}",test,0,出張 で お 世話 に なる ます た 。
1,pn17q00002,neutral,朝食は普通でした。,"{""0"": 3}",test,0,朝食 は 普通 です た 。
2,pn17q00003,positive,また是非行きたいです。,"{""1"": 3}",test,0,また 是非 行く たい です 。
3,pn17q00004,positive,また利用したいと思えるホテルでした。,"{""1"": 3}",test,0,また 利用 する たい と 思える ホテル です た 。
4,pn17q00005,positive,駅から近くて便利でした。,"{""0"": 1, ""1"": 2}",test,0,駅 から 近い て 便利 です た 。


In [15]:
save_columns = ["text", "label_num", "tokens"]

pn_df[judges_all_same] \
    .reset_index(drop=True)[save_columns] \
    .to_csv("input/pn_same_judge_preprocessed.csv", index=None)

In [16]:
df = pd.read_csv("input/pn_same_judge_preprocessed.csv")